In [2]:
!uv pip install google-genai python-dotenv

Audited 2 packages in 2ms


In [ ]:
import os
from typing import List

from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

True

In [7]:
client = genai.Client(api_key=os.getenv("FREE_STUDIO_KEY"))

with open("./system_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

In [ ]:
# Define the response schema for structured output
response_schema = types.Schema(
    type=types.Type.OBJECT,
    properties={
        "financial_metrics": types.Schema(
            type=types.Type.ARRAY, items=types.Schema(type=types.Type.STRING)
        ),
        "risk_factors": types.Schema(
            type=types.Type.ARRAY, items=types.Schema(type=types.Type.STRING)
        ),
        "business_outlook": types.Schema(type=types.Type.STRING),
    },
    required=["financial_metrics", "risk_factors", "business_outlook"],
)

In [ ]:
def create_batch_request(prompts: List[str]):
    """
    Create a batched request using the same system prompt and return the batch job.
    """
    # Prepare the requests
    requests = []
    for prompt in prompts:
        request = {
            "contents": [types.Content(role="user", parts=[types.Part(text=prompt)])],
            "config": types.GenerateContentConfig(
                system_instruction=system_prompt,
                response_mime_type="application/json",
                response_schema=response_schema,
            ),
        }
        requests.append(request)

    # Create the batch job
    batch_job = client.batches.create(
        model="gemini-1.5-pro",
        src=requests,
    )
    return batch_job


# Example usage
prompts = [
    "Analyze this 10-K filing excerpt: [insert excerpt here]",
    "Analyze this 10-Q filing excerpt: [insert excerpt here]",
    # Add more prompts as needed
]

# Create the batch job
batch_job = create_batch_request(prompts)
print(f"Batch job created: {batch_job.name}")

# To get results, you can poll the job
# batch_job = client.batches.get(name=batch_job.name)
# When completed, results are in batch_job.dest if specified, or use client.batches.list() to get output